# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Experiment, Model, Webservice
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep

import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
We will be using the 'Credit Card customers' dataset from Kaggle and predicting if a customer is likely to churn.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# Choose a name for experiment
experiment_name = 'demo-automl-experiment'

experiment=Experiment(ws, experiment_name)

# Prepare the datastore to upload data
datastore = ws.get_default_datastore()

# Set the upload location to target_path in datastore
datastore.upload(src_dir='./', target_path = 'data')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E7H6GEPL8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Uploading an estimated of 7 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 7
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 2 files out of an estimated total of 7
Uploading ./hyperparameter_tuning.ipynb.amltemp
Uploaded ./hyperparameter_tuning.ipynb.amltemp, 3 files out of an estimated total of 7
Uploading ./score.py
Uploaded ./score.py, 4 files out of an estimated total of 7
Uploading ./train.py
Uploaded ./train.py, 5 files out of an estimated total of 7
Uploading ./.ipynb_aml_checkpoints/automl-checkpoint2021-0-11-12-1-1.ipynb
Uploaded ./.ipynb_aml_checkpoints/a

In [5]:
# Choose a name for CPU cluster
compute_name = "demo-cluster"

# Verify that cluster does not already exist
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


In [6]:
dataset = Dataset.get_by_name(ws, name = "bankchurners")
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/01-11-2021_120232_UTC/BankChurners.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "e5d3d8fa-0441-4c59-b016-7b1cfe8e003c",
    "name": "bankchurners",
    "version": 1,
    "description": "bc dataset",
    "workspace": "Workspace.create(name='quick-starts-ws-134326', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-134326')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# List the automl config
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = "classification",
    compute_target = "demo-cluster",
    max_concurrent_iterations = 5,
    training_data = dataset,
    primary_metric = "accuracy",
    label_column_name = "Attrition_Flag",
    n_cross_validations = 3,
    enable_early_stopping = True
)

In [8]:
# Submit the experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on demo-cluster with default configuration
Running on remote compute: demo-cluster
Parent Run ID: AutoML_5946d9b6-159a-4cd8-ac46-aa146cbd3c4e


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output = True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1627                             |Attrited Customer                |10127                                 |
+---------------------------------+----------

{'runId': 'AutoML_5946d9b6-159a-4cd8-ac46-aa146cbd3c4e',
 'target': 'demo-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-11T12:05:45.893505Z',
 'endTimeUtc': '2021-01-11T12:23:54.737669Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'demo-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"demo-automl-experiment","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-134326","workspace_name":"quick-starts-ws-134326","region":"southcentralus","compute_target":"demo-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_clas

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
# Retrieve the best AutoML model
best_run, best_model = remote_run.get_output()
best_metrics = best_run.get_metrics()

In [14]:
#Save the best model
model = best_run.register_model(model_name = "best_automl_model", model_path = "./outputs/")
print(best_run)

Run(Experiment: demo-automl-experiment,
Id: AutoML_5946d9b6-159a-4cd8-ac46-aa146cbd3c4e_38,
Type: azureml.scriptrun,
Status: Completed)


In [15]:
print(best_metrics["accuracy"])
print(best_model._final_estimator)

0.9706723714235562
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                                                

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.automl.core.shared import constants

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

# Create the inference config

inference_config = InferenceConfig(entry_script='score.py', environment=myenv)


In [16]:
# Deploy the model as a webservice
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    auth_enabled=True,
    memory_gb=1,
    tags = {'name':'bank churners'},
    description='bank churners model'
)
service = Model.deploy(workspace=ws,
                        name = 'model-service',
                        models= [model],
                        inference_config=inference_config,
                        deployment_config=aci_config
)

# Wait until deployment is complete
service.wait_for_deployment(show_output = True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................................................................................................................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 089fee0c-9ae9-4d3a-921d-7ba14c60d905
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: model-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image b582091ad3e740f2b06732eec062c8b2.azurecr.io/azureml/azureml_b78ed8698c008e188485d4f8dc0ae5ff locally. Please refer to https://aka.ms/debugimage#service

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 089fee0c-9ae9-4d3a-921d-7ba14c60d905
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: model-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image b582091ad3e740f2b06732eec062c8b2.azurecr.io/azureml/azureml_b78ed8698c008e188485d4f8dc0ae5ff locally. Please refer to https://aka.ms/debugim

In [17]:
print(service.get_logs)

<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='quick-starts-ws-134326', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-134326'), name=model-service, image_id=None, compute_type=None, state=ACI, scoring_uri=Failed, tags=None, properties={'name': 'bank churners'}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})>


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json

data_sample = dataset.to_pandas_dataframe().sample(3)
test_sample = json.dumps({'data':data_sample.to_dict(orient='records')})
print(test_sample)


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Print web service logs
service_logs = service.get_logs()

for line in service_logs.split('\n'):
    print(line)

# Delete the service
service.delete()